In [1]:
import tensorflow as tf
class DenseLayer:
    def __init__(self, input_size, output_size, activation='sigmoid'):
        self.weights = tf.Variable(tf.random.normal((input_size, output_size), stddev=0.01))
        self.biases = tf.Variable(tf.zeros(output_size))
   
    def forward(self, inputs):
        self.inputs = inputs
        self.output = tf.matmul(self.inputs, self.weights) + self.biases
        return self.output
    
    def backward(self, grad_output, learning_rate):
        with tf.GradientTape(persistent=True) as tape:
            output = tf.matmul(self.inputs, self.weights) + self.biases
            loss = tf.reduce_mean(tf.square(output - grad_output))
        
        grad_weights = tape.gradient(loss, self.weights)
        grad_biases = tape.gradient(loss, self.biases)
        grad_input = tape.gradient(loss, self.inputs)
        
        self.weights.assign_sub(learning_rate * grad_weights)
        self.biases.assign_sub(learning_rate * grad_biases)
        
        return grad_input


In [2]:
class DenseNetwork:
    def __init__(self):
        self.layers = []
    
    def add_layer(self, layer):
        self.layers.append(layer)
    
    def forward(self, inputs):
        for layer in self.layers:
            inputs = layer.forward(inputs)
        return inputs
    
    def backward(self, grad_output, learning_rate):
        for layer in reversed(self.layers):
            grad_output = layer.backward(grad_output, learning_rate)


In [6]:
# Train the DenseNetwork using gradient descent

from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import numpy as np

# Generate synthetic dataset
X, y = make_regression(n_samples=100, n_features=10, noise=0.5, random_state=42)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)

# Standardize the input features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


learning_rate = 0.001
num_epochs = 1000
dense_net = DenseNetwork()
for epoch in range(num_epochs):
    # Forward pass
    with tf.GradientTape() as tape:
        y_pred = dense_net.forward(X_train_scaled)
        loss = tf.reduce_mean(tf.square(y_pred - y_train))
    
    # Backward pass
    grad_output = 2 * (y_pred - y_train) / len(X_train_scaled)
    dense_net.backward(grad_output, learning_rate)


In [8]:
# Predict with the DenseNetwork
y_pred_dense = dense_net.forward(X_test_scaled)
y_test - y_pred_dense
# Compare the results
mse = mean_squared_error(y_test, y_pred_dense)
print("Mean Squared Error (DenseNetwork implemented from scratch):", mse)


ValueError: y_true and y_pred have different number of output (1!=10)